## Project Description: Next Word Prediction Using LSTM
#### Project Overview:

This project aims to develop a deep learning model for predicting the next word in a given sequence of words. The model is built using Long Short-Term Memory (LSTM) networks, which are well-suited for sequence prediction tasks. The project includes the following steps:

1- Data Collection: We use the text of Shakespeare's "Hamlet" as our dataset. This rich, complex text provides a good challenge for our model.

2- Data Preprocessing: The text data is tokenized, converted into sequences, and padded to ensure uniform input lengths. The sequences are then split into training and testing sets.

3- Model Building: An LSTM model is constructed with an embedding layer, two LSTM layers, and a dense output layer with a softmax activation function to predict the probability of the next word.

4- Model Training: The model is trained using the prepared sequences, with early stopping implemented to prevent overfitting. Early stopping monitors the validation loss and stops training when the loss stops improving.

5- Model Evaluation: The model is evaluated using a set of example sentences to test its ability to predict the next word accurately.

6- Deployment: A Streamlit web application is developed to allow users to input a sequence of words and get the predicted next word in real-time.

## Data Collection

In [ ]:
import nltk
import pandas as pd
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


## Data Preprocessing

In [ ]:
# Credits for data: https://www.gutenberg.org/files/1661/1661-0.txt
with open('1661-0.txt', 'r', encoding='utf-8') as file:
    text = file.read()

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

In [ ]:
list(tokenizer.word_index.items())[:10]

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
input_sequences[:10]

In [ ]:
# Pad Sequences
max_sequence_len = max([len(x) for x in input_sequences])

In [ ]:
max_sequence_len

In [ ]:
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
input_sequences

In [ ]:
# Split the data into predictors and label
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
X

In [ ]:
y

In [ ]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
# Split train test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU

model = Sequential([
    Embedding(input_dim = total_words, output_dim = 100, input_length=max_sequence_len-1),
    LSTM(150, return_sequences=True),
    Dropout(0.4),  # Increased dropout
    LSTM(100),
    Dense(100, activation='relu'),
    Dropout(0.4),  # Increased dropout
    Dense(total_words, activation='softmax')
])


In [ ]:
from tensorflow.keras.optimizers import Adam

# Use a much smaller learning rate
optimizer = Adam(learning_rate=0.0001) 

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    verbose=1,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

In [ ]:
model.summary()

## GRU

In [ ]:
model_gru = Sequential(
    [
        Embedding(total_words, 100, input_length=max_sequence_len-1),
        GRU(150, return_sequences=True),
        Dropout(0.2),
        GRU(100),
        Dense(total_words, activation='softmax')
    ]
)
model_gru.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
history_gru = model_gru.fit(
    X_train,
    y_train,
    epochs=20,
    verbose=1,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

In [ ]:
def predict_next_word(model, tokenizer, text_sequence, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text_sequence])[0]

    if len(token_list) >= max_sequence_len - 1:
        token_list = token_list[-(max_sequence_len - 1):]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)[0]

    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [ ]:
input_text = "I will leave if they"
predicted_word = predict_next_word(model_gru, tokenizer, input_text, max_sequence_len)
print(f"Input: '{input_text}' -> \nPredicted next word: '{predicted_word}'")

In [ ]:
model_gru.save('next_word_gru_model.h5')

# Save the model
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)